# Tourmaline Meta-analysis Notebook

This notebook shows how to take Tourmaline outputs from two (or more) separate analyses and merge them for a meta-analysis. The two analyses are derived from the test data that comes with Tourmaline, and the output must first be generated (commands provided).

## Import packages

In [1]:
from qiime2 import Artifact
from qiime2 import Visualization
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## Set file paths

**The value of `tourmdir` should be changed if this notebook is not being run from its original location in the `notebooks` directory.**

In [7]:
tourmdir = '..'

In [8]:
dir_analysis1 = '%s/meta-analysis/analysis1' % tourmdir
dir_analysis2 = '%s/meta-analysis/analysis2' % tourmdir

## Set up Tourmaline analyses

The below commands will, for each analysis, clone the Tourmaline respository, initialize it using parameters and files from the top-level repository (already downloaded), and copy the provided metadata and manifest files to their proper location in `00-data`.

In [3]:
%%bash
cd ../meta-analysis/analysis1
git clone https://github.com/aomlomics/tourmaline
cd tourmaline
./scripts/initialize_dir_from_existing_tourmaline_dir.sh ../../..
cp ../metadata.tsv 00-data
cp ../manifest_* 00-data
cp ../../../01-imported/silva-138-99-seqs-515-806.qza 01-imported
cp ../../../01-imported/silva-138-99-tax-515-806.qza 01-imported

Cloning into 'tourmaline'...


Removed:
00-data/fastq/Sample_134723_R2.fastq.gz
00-data/fastq/Sample_134722_R1.fastq.gz
00-data/fastq/Sample_134789_R2.fastq.gz
00-data/fastq/Sample_134719_R1.fastq.gz
00-data/fastq/Sample_134768_R2.fastq.gz
00-data/fastq/Sample_134718_R2.fastq.gz
00-data/fastq/Sample_134771_R2.fastq.gz
00-data/fastq/Sample_134790_R2.fastq.gz
00-data/fastq/Sample_134778_R2.fastq.gz
00-data/fastq/Sample_134767_R2.fastq.gz
00-data/fastq/Sample_134717_R2.fastq.gz
00-data/fastq/Sample_134716_R1.fastq.gz
00-data/fastq/Sample_134772_R2.fastq.gz
00-data/fastq/Sample_134777_R2.fastq.gz
00-data/fastq/Sample_134721_R1.fastq.gz
00-data/fastq/Sample_134720_R2.fastq.gz
00-data/fastq/Sample_134778_R1.fastq.gz
00-data/fastq/Sample_134771_R1.fastq.gz
00-data/fastq/Sample_134790_R1.fastq.gz
00-data/fastq/Sample_134789_R1.fastq.gz
00-data/fastq/Sample_134719_R2.fastq.gz
00-data/fastq/Sample_134768_R1.fastq.gz
00-data/fastq/Sample_134718_R1.fastq.gz
00-data/fastq/Sample_134723_R1.fastq.gz
00-data/fastq/Sample_134722_R2.

In [4]:
%%bash
cd ../meta-analysis/analysis2
git clone https://github.com/aomlomics/tourmaline
cd tourmaline
./scripts/initialize_dir_from_existing_tourmaline_dir.sh ../../..
cp ../metadata.tsv 00-data
cp ../manifest_* 00-data
cp ../../../01-imported/silva-138-99-seqs-515-806.qza 01-imported
cp ../../../01-imported/silva-138-99-tax-515-806.qza 01-imported

Cloning into 'tourmaline'...


Removed:
00-data/fastq/Sample_134723_R2.fastq.gz
00-data/fastq/Sample_134722_R1.fastq.gz
00-data/fastq/Sample_134789_R2.fastq.gz
00-data/fastq/Sample_134719_R1.fastq.gz
00-data/fastq/Sample_134768_R2.fastq.gz
00-data/fastq/Sample_134718_R2.fastq.gz
00-data/fastq/Sample_134771_R2.fastq.gz
00-data/fastq/Sample_134790_R2.fastq.gz
00-data/fastq/Sample_134778_R2.fastq.gz
00-data/fastq/Sample_134767_R2.fastq.gz
00-data/fastq/Sample_134717_R2.fastq.gz
00-data/fastq/Sample_134716_R1.fastq.gz
00-data/fastq/Sample_134772_R2.fastq.gz
00-data/fastq/Sample_134777_R2.fastq.gz
00-data/fastq/Sample_134721_R1.fastq.gz
00-data/fastq/Sample_134720_R2.fastq.gz
00-data/fastq/Sample_134778_R1.fastq.gz
00-data/fastq/Sample_134771_R1.fastq.gz
00-data/fastq/Sample_134790_R1.fastq.gz
00-data/fastq/Sample_134789_R1.fastq.gz
00-data/fastq/Sample_134719_R2.fastq.gz
00-data/fastq/Sample_134768_R1.fastq.gz
00-data/fastq/Sample_134718_R1.fastq.gz
00-data/fastq/Sample_134723_R1.fastq.gz
00-data/fastq/Sample_134722_R2.

After running the above steps, check that the required input files are all present.

## Run Tourmaline analyses

Run the two commands below from your Terminal (i.e., outside of this notebook). Each command will take ~5 minutes to complete.

```bash
conda activate snakemake
cd ../meta-analysis/analysis1/tourmaline
snakemake --use-conda dada2_pe_report_unfiltered --cores 4
```

```bash
conda activate snakemake
cd ../meta-analysis/analysis2/tourmaline
snakemake --use-conda dada2_pe_report_unfiltered --cores 4
```

## Merge outputs

### Merge feature tables

Additional feature table files can be merged by adding more `--i-tables` inputs.

In [ ]:
%%bash
qiime feature-table merge \
--i-tables ../meta-analysis/analysis1/tourmaline/02-output-dada2-pe-unfiltered/00-table-repseqs/table.qza \
--i-tables ../meta-analysis/analysis2/tourmaline/02-output-dada2-pe-unfiltered/00-table-repseqs/table.qza \
--o-merged-table ../meta-analysis/merged_table.qza

### Merge representative sequences

Additional representative sequences files can be merged by adding more `--i-data` inputs.

In [ ]:
%%bash
qiime feature-table merge-seqs \
--i-data ../meta-analysis/analysis1/tourmaline/02-output-dada2-pe-unfiltered/00-table-repseqs/repseqs.qza \
--i-data ../meta-analysis/analysis2/tourmaline/02-output-dada2-pe-unfiltered/00-table-repseqs/repseqs.qza \
--o-merged-data ../meta-analysis/merged_repseqs.qza

### Merge taxonomies

Additional taxonomy files can be merged by adding more `--i-data` inputs.

In [ ]:
%%bash
qiime feature-table merge-taxa \
--i-data ../meta-analysis/analysis1/tourmaline/02-output-dada2-pe-unfiltered/01-taxonomy/taxonomy.qza \
--i-data ../meta-analysis/analysis2/tourmaline/02-output-dada2-pe-unfiltered/01-taxonomy/taxonomy.qza \
--o-merged-data ../meta-analysis/merged_taxonomy.qza

### Merge metadata

Additional metadata files can be merged by importing more files and adding them to the `concat` command.

In [ ]:
metadata1 = pd.read_csv('%s/tourmaline/00-data/metadata.tsv' % dir_analysis1, sep='\t', index_col=0)
metadata2 = pd.read_csv('%s/tourmaline/00-data/metadata.tsv' % dir_analysis2, sep='\t', index_col=0)
metadata_merged = pd.concat([metadata1, metadata2])
metadata_merged.to_csv('../meta-analysis/merged_metadata.tsv', sep='\t')

## Analyze merged output

The commands below are some initial examples of analyzing the merged output. A full meta-analysis will go much deeper than this.

### Taxonomy barplot

Create a taxonomy barplot.

In [ ]:
%%bash
qiime taxa barplot \
--i-table ../meta-analysis/merged_table.qza \
--i-taxonomy ../meta-analysis/merged_taxonomy.qza \
--m-metadata-file ../meta-analysis/merged_metadata.tsv \
--o-visualization ../meta-analysis/merged_taxa_barplot.qzv

### Alpha diversity

Create an alpha diversity vector using the 'shannon' metric.

In [ ]:
%%bash
qiime diversity alpha \
--i-table ../meta-analysis/merged_table.qza \
--p-metric shannon \
--o-alpha-diversity ../meta-analysis/merged_alpha_shannon.qza

### Beta diversity

Create a beta diversity distance matrix using the 'braycurtis' metric.

In [ ]:
%%bash
qiime diversity beta \
--i-table ../meta-analysis/merged_table.qza \
--p-metric braycurtis \
--o-distance-matrix ../meta-analysis/merged_beta_braycurtis.qza